In [22]:
import pandas as pd
import matplotlib.pyplot as plt
import os

def load_data(ticker):
    dir = os.path.join("Database", 'US', f"{ticker}-1d-5.csv")
    df = pd.read_csv(dir, index_col=0)
    df.index = pd.to_datetime(df.index, utc = True)
    df.index = df.index.tz_convert('US/Eastern')

    return df


def fact(df):
    retrurns = df['Close'].pct_change()

    mean = retrurns.mean()
    std = retrurns.std()
    vol = std * (252 ** 0.5)
    skew = retrurns.skew()
    kurt = retrurns.kurtosis() - 3

    VaR = retrurns.quantile(0.05)
    CVaR = retrurns[retrurns <= VaR].mean()

    sharpe = mean / std * (252 ** 0.5)

    return {
        'Mean': mean,
        'Volatility': vol,
        'Skewness': skew,
        'Kurtosis': kurt,
        'VaR': VaR,
        'CVaR': CVaR,
        'Sharpe': sharpe
    }


In [32]:
holder = {}
AllStockPrices = {}
for ticker in ["^GSPC", 'AAPL', 'COP', 'GOOG', 'AMZN', 'LOW', 'COP']:
    df = load_data(ticker)
    AllStockPrices[ticker] = df
    holder[ticker] = fact(df)

df = pd.DataFrame(holder)
print(df)

                ^GSPC      AAPL      MSFT      GOOG      AMZN        LOW  \
Mean         0.000495  0.001405  0.001290  0.000936  0.000773   0.000949   
Volatility   0.193195  0.317461  0.302844  0.318204  0.349240   0.331531   
Skewness    -0.541238  0.025774 -0.016821 -0.086303  0.089869  -1.377789   
Kurtosis    11.706135  2.141042  4.308589  1.135071  1.418543  18.547293   
VaR         -0.018121 -0.030693 -0.028568 -0.030375 -0.032481  -0.028969   
CVaR        -0.029960 -0.044994 -0.042358 -0.046506 -0.049331  -0.049579   
Sharpe       0.645873  1.115587  1.073138  0.741644  0.557986   0.721566   

                  COP  
Mean         0.000962  
Volatility   0.446546  
Skewness     0.037390  
Kurtosis    10.337935  
VaR         -0.037258  
CVaR        -0.061943  
Sharpe       0.542944  


In [33]:
import statsmodels.api as sm


snp = AllStockPrices['^GSPC']
cop = AllStockPrices['COP']

snp['Return'] = snp['Close'].pct_change()
cop['Return'] = cop['Close'].pct_change()

snp = snp.dropna()
cop = cop.dropna()

# Join the two dataframes along the axis=1
df = pd.concat([cop['Return'], snp['Return']], axis=1)
df.columns = ['COP', 'S&P']

df.dropna(inplace=True)

# Make a single index model and print the results
X = df['S&P']
y = df['COP']

X1 = sm.add_constant(X)

model = sm.OLS(y, X1).fit()
print(model.summary())


                            OLS Regression Results                            
Dep. Variable:                    COP   R-squared:                       0.296
Model:                            OLS   Adj. R-squared:                  0.295
Method:                 Least Squares   F-statistic:                     509.4
Date:                Fri, 23 Feb 2024   Prob (F-statistic):           1.90e-94
Time:                        16:09:27   Log-Likelihood:                 2809.7
No. Observations:                1214   AIC:                            -5615.
Df Residuals:                    1212   BIC:                            -5605.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0004      0.001      0.551      0.5

In [36]:
import statsmodels.api as sm


snp = AllStockPrices['^GSPC']
cop = AllStockPrices['COP']

large_cap = AllStockPrices['AAPL']
small_cap = AllStockPrices['LOW']

snp['Return'] = snp['Close'].pct_change()
cop['Return'] = cop['Close'].pct_change()
large_cap['Return'] = large_cap['Close'].pct_change()
small_cap['Return'] = small_cap['Close'].pct_change()
smb = small_cap['Return'] - large_cap['Return']

snp = snp.dropna()
cop = cop.dropna()
large_cap = large_cap.dropna()
small_cap = small_cap.dropna()
smb = smb.dropna()

# Join the two dataframes along the axis=1
df = pd.concat([cop['Return'], snp['Return'], smb], axis=1)
df.columns = ['COP', 'S&P', 'SMB']

df.dropna(inplace=True)

# Make a single index model and print the results
X = df[['S&P', 'SMB']]
y = df['COP']

X1 = sm.add_constant(X)

model = sm.OLS(y, X1).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                    COP   R-squared:                       0.306
Model:                            OLS   Adj. R-squared:                  0.305
Method:                 Least Squares   F-statistic:                     266.6
Date:                Fri, 23 Feb 2024   Prob (F-statistic):           1.16e-96
Time:                        16:12:45   Log-Likelihood:                 2818.2
No. Observations:                1214   AIC:                            -5630.
Df Residuals:                    1211   BIC:                            -5615.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0004      0.001      0.637      0.5